<a href="https://colab.research.google.com/github/1kaiser/Snow-cover-area-estimation/blob/main/OpenAltimetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAltimetry ICESat-2: Elevation Workflow
Overview


This notebook pulls data from ICESat-2 surface-specific products - ATL06, ATL07, ATL08, ATL10, ATL12, ATL13 via the OpenAltimetry API.


Instructions
Replace the "OA_API_URL" string with the data copied from : "Copy API URL" from the OpenAltimetry elevation chart and start visualizing your data!
Author: Minh Phan - mnphan@ucsd.edu. Based off the work of Luis Lopez - luis.lopez@nsidc.org

#**ALT06 LAND ICE ELEVATION**

In [ ]:
import json
import requests

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447


OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

print(OA_API_URL)

r = requests.get(OA_API_URL)
data = r.json()


In [ ]:
data

**[dates and track id google sheet](https://docs.google.com/spreadsheets/d/13sjRCD2E2oyjkJwXOxeu5P3V6Ql9nW0KZe1gnsPFNOM/edit?usp=sharing)**


![image](https://user-images.githubusercontent.com/26379748/227423479-8e35d0da-3617-4c01-bf77-8074ebeb14e2.png)


In [ ]:
a = [["2019-02-09",653],
["2019-02-13",714],
["2019-02-24",889],
["2019-02-28",950],
["2019-05-11",653],
["2019-05-15",714],
["2019-05-26",889],
["2019-05-30",950],
["2019-08-09",653],
["2019-08-13",714],
["2019-08-25",889],
["2019-08-29",950],
["2019-11-08",653],
["2019-11-12",714],
["2019-11-24",889],
["2019-11-28",950],]

In [ ]:
len(a)

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot
from pathlib import Path  



filepath = Path('2019PALL.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

# Bounding box coordinates
left   = 75.979728
right  = 77.866667
bottom = 31.453599
top    = 32.416667

date = '2019-10-26'
trackid = 447

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []
complete_dataset = []
# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA >>> ',b_url)
    r = requests.get(b_url)
    data = r.json()
    return data


b =[]
for i in range(len(a)):
  OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl06?date='+a[i][0]+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(a[i][1])+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'
  OA_API_URL += '&client=jupyter';
  b.append(getData(OA_API_URL))

for elevation_data in b:
  product = elevation_data["product"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['beam'] for word in OA_BEAMS):
          series = []
          
          data_name = 'lat_lon_elev'
          if product == 'ATL08':
              data_name = 'lat_lon_elev_canopy'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2]
              })
              
          print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['beam']
              df_beams.append(df)


 
              
  print('df_beams len: ' + str(len(df_beams)))

# <<<<SAVING TO .csv FILE>>>>
pd.concat(df_beams).to_csv(filepath)

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


In [ ]:
pd.concat(df_beams)


https://openaltimetry.org

In [ ]:
a = !curl -X GET "https://openaltimetry.org/data/api/icesat2/atl06?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&trackId=447&client=portal&outputFormat=json" -H "accept: */*"

In [ ]:
a

In [ ]:
!wget https://openaltimetry.org/data/api/icesat2/getTracks?date=2019-10-26&minx=75.979728&miny=31.453599&maxx=77.866667&maxy=32.416667&outputFormat=json -O a.json


**VISUALIZATION**

In [ ]:
a = np.asarray([[	"	A	"	,	"	10/23/2018	"	,	]	,
[	"	B	"	,	"	10/27/2018	"	,	]	,
[	"	C	"	,	"	11/6/2018	"	,	]	,
[	"	D	"	,	"	11/10/2018	"	,	]	,
[	"	E	"	,	"	11/14/2018	"	,	]	,
[	"	F	"	,	"	11/25/2018	"	,	]	,
[	"	G	"	,	"	11/29/2018	"	,	]	,
[	"	H	"	,	"	12/3/2018	"	,	]	,
[	"	I	"	,	"	12/5/2018	"	,	]	,
[	"	J	"	,	"	12/13/2018	"	,	]	,
[	"	K	"	,	"	12/24/2018	"	,	]	,
[	"	L	"	,	"	12/28/2018	"	,	]	,
[	"	M	"	,	"	1/1/2019	"	,	]	,
[	"	N	"	,	"	1/3/2019	"	,	]	])

In [ ]:
from numpy.core.fromnumeric import transpose
a = transpose(a)
a.shape

In [ ]:
b = np.ones((4,10))+np.ones((4,10))
b.shape
b

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data1 = a

colors1 = ['C{}'.format(i) for i in range(4)]

lineoffsets1 = [-4, -3]
linelengths1 = [1, 2 ]

plt.eventplot(data1, colors=colors1, lineoffsets=lineoffsets1, linelengths=linelengths1)
plt.title('Event Plot')
plt.xlabel('Time(s)')
plt.ylabel('Signal')

#**ALT08 GROUND AND CANOPY HEIGHT**

In [ ]:
import json
import requests

# Bounding box coordinates
left   = 77.301555
right  = 77.637165
bottom = 31.779603
top    = 32.041187

date = '2019-02-24'
trackid = 889


OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl08?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

print(OA_API_URL)

r = requests.get(OA_API_URL)
data = r.json()


In [ ]:
data

In [ ]:
a = [["2019-05-26",889],]

In [ ]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot
from pathlib import Path  

filepath = Path('889MAYALT08_2019_MS.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

# Bounding box coordinates
left   = 77.301555
right  = 77.637165
bottom = 31.779603
top    = 32.041187


date = '2019-10-26'
trackid = 447

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl08?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'

OA_API_URL += '&client=jupyter';

OA_BEAMS = ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']

df_beams = []
oa_plots = []
complete_dataset = []
# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA >>> ',b_url)
    r = requests.get(b_url)
    data = r.json()
    return data


b =[]
for i in range(len(a)):
  OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl08?date='+a[i][0]+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(a[i][1])+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'
  OA_API_URL += '&client=jupyter';
  b.append(getData(OA_API_URL))

for elevation_data in b:
  product = elevation_data["product"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['beam'] for word in OA_BEAMS):
          series = []
          
          data_name = 'lat_lon_elev'
          if product == 'ATL08':
              data_name = 'lat_lon_elev_canopy'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2],
                  'Canopy': p[3]
              })
              
          print ('beam ' + series_beam['beam'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['beam']
              df_beams.append(df)


 
              
  print('df_beams len: ' + str(len(df_beams)))

# <<<<SAVING TO .csv FILE>>>>
pd.concat(df_beams).to_csv(filepath)

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
                                    marker=dict(
                                        size=2,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ),
                                    line=dict(
                                        color='darkblue',
                                        width=2
                                    )
                                )
                   )

layout = go.Layout(
    width=1200,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)

iplot(fig)


#**ALT03 RAW DATA**

In [ ]:
import json
import requests

# Bounding box coordinates
left   = 77.301555
right  = 77.637165
bottom = 31.779603
top    = 32.041187

date = '2019-02-24'
trackid = 889


OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json'

OA_API_URL += '&client=jupyter';

print(OA_API_URL)

r = requests.get(OA_API_URL)
data = r.json()


In [ ]:
curl -X 'GET' \
  'https://openaltimetry.org/data/api/icesat2/atl03?date=2019-02-24&miny=31.779603&maxy=32.041187&minx=77.301555&maxx=77.637165&beamName=gt2l&trackId=889&client=portal&photonConfidence=high&sampling=true&outputFormat=json' \
  -H 'accept: */*'

In [ ]:
[{'beam_name': 'gt3r',
  'total_photon_count': 70297048,
  'select_photon_count': 70303,
  'percentage': 0.1,
  'series': [{'name': 'Noise',
    'photon_count': 33076,
    'data': [[15.513679430657804, -95.6148075025523, 15.525837],
     [15.541376507563745, -95.617650126665, 7.1815844],
     [15.568710564677138, -95.62046201300774, 17.62772],
     [15.577618987872631, -95.62138089111858, 12.514256],
     [15.586719582354583, -95.62231583168484, 18.253971],

{'product': 'ATL08',
 'scForward': False,
 'trackId': 889,
 'series': [{'beam': 'gt3r',
   'isStrongBeam': False,
   'lat_lon_elev_canopy': [[32.040348052978516, 77.46642303466797, 3215.094,8.703613],
    [32.03944778442383, 77.46632385253906, 3111.0308, 13.845459],
    [32.03854751586914, 77.46621704101562, 3076.8823, 10.201416],
    [32.03764724731445, 77.46611785888672, 3056.3738, 13.573242],

OA_BEAMS = ['noise', 'buffer', 'low', 'medium', 'high']

for elevation_data in b:
  product = elevation_data["beam_name"]

  print('Reading data product: ' + product)

  for series_beam in elevation_data['series']:
      if any(word in series_beam['name'] for word in OA_BEAMS):
          series = []
          
          data_name = 'data'

          for p in series_beam[data_name]:
              series.append({
                  'lat': p[0],
                  'lon': p[1],
                  'h': p[2]
              })
              
          print ('beam ' + series_beam['name'] + ' - len = ' + str(len(series)))

          if (len(series) > 0):
              df = pd.DataFrame.from_dict(series)
              df.name = series_beam['name']
              df_beams.append(df)



In [ ]:
data

In [65]:
a = [["2019-05-26",889],]

In [66]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot
from pathlib import Path  

filepath = Path('ALT08_2022_MS.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

# Bounding box coordinates
left   = 77.301555
right  = 77.637165
bottom = 31.779603
top    = 32.041187

date = '2019-05-26'
trackid = 889

# We populate a list with the elevation data using the OpenAltimetry API
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+date+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(trackid)+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=true&outputFormat=json'

OA_API_URL += '&client=jupyter';


df_beams = []
oa_plots = []
complete_dataset = []
# This function will request the 6 tracks using OpenAltimetry's API
def getData(url):
    series = []
    b_url = url

    print('Requesting data from OA >>> ',b_url)
    r = requests.get(b_url)
    data = r.json()
    return data


b =[]
for i in range(len(a)):
  OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?date='+a[i][0]+'&minx='+str(left)+'&miny='+str(bottom)+'&maxx='+str(right)+'&maxy='+str(top)+'&trackId='+str(a[i][1])+'&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=false&outputFormat=json'
  OA_API_URL += '&client=jupyter';
  b.append(getData(OA_API_URL))

OA_BEAMS = ['Noise', 'Buffer', 'Low', 'Medium', 'High']
df_beams = []

for elevation_data in b[0]:
  product = elevation_data['beam_name']

  print('\n Reading data product: ' + product)
  for series_beam in elevation_data['series']:
    if any(word in series_beam['name'] for word in OA_BEAMS):
      series = []
      
      data_name = 'data'

      for p in series_beam[data_name]:
          series.append({
              'lat': p[0],
              'lon': p[1],
              'h': p[2]
          })
          
      print ('beam ' + series_beam['name'] + ' - len = ' + str(len(series)))

      if (len(series) > 0):
          df = pd.DataFrame.from_dict(series)
          print(len(df))
          df = df[(df.lon >= left) & (df.lon <= right) & (df.lat >= bottom) & (df.lat <= top)]
          print(len(df))
          df.name = series_beam['name']
          df_beams.append(df) 
              
  print('df_beams len: ' + str(len(df_beams)))

# <<<<SAVING TO .csv FILE>>>>
pd.concat(df_beams).to_csv(filepath)

# for df in df_beams:
#     oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'],
#                                     marker=dict(
#                                         size=2,
#                                         color=df['h'],
#                                         colorscale='Viridis',   # choose a colorscale
#                                         opacity=0.8
#                                     ),
#                                     line=dict(
#                                         color='darkblue',
#                                         width=2
#                                     )
#                                 )
#                    )

# layout = go.Layout(
#     width=1200,
#     height=600,
#     scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
#                  xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
# )

# print('Plotting...')

# fig = go.Figure(data=oa_plots, layout=layout)

# iplot(fig)


Requesting data from OA >>>  https://openaltimetry.org/data/api/icesat2/atl03?date=2019-05-26&minx=77.301555&miny=31.779603&maxx=77.637165&maxy=32.041187&trackId=889&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&sampling=false&outputFormat=json&client=jupyter

 Reading data product: gt3r
beam Noise - len = 49029
49029
49029
beam Buffer - len = 17
17
17
beam Low - len = 534
534
534
beam Medium - len = 4
4
4
beam High - len = 0
df_beams len: 4

 Reading data product: gt3l
beam Noise - len = 49307
49307
49306
beam Buffer - len = 14
14
14
beam Low - len = 969
969
969
beam Medium - len = 365
365
365
beam High - len = 61
61
61
df_beams len: 9

 Reading data product: gt2r
beam Noise - len = 48320
48320
48320
beam Buffer - len = 13
13
13
beam Low - len = 954
954
954
beam Medium - len = 19
19
19
beam High - len = 21
21
21
df_beams len: 14

 Reading data product: gt2l
beam Noise - len = 49341
49341
49329
beam Buffer - len = 23
23
23
beam Low - len = 813
813


In [ ]:
b[0][0]['series'][1]['name']

In [ ]:
OA_BEAMS = ['Noise', 'Buffer', 'Low', 'Medium', 'High']
df_beams = []

for elevation_data in b[0]:
  product = elevation_data['beam_name']

  print('\n Reading data product: ' + product)
  for series_beam in elevation_data['series']:
    if any(word in series_beam['name'] for word in OA_BEAMS):
      series = []
      
      data_name = 'data'

      for p in series_beam[data_name]:
          series.append({
              'lat': p[0],
              'lon': p[1],
              'h': p[2]
          })
          
      print ('beam ' + series_beam['name'] + ' - len = ' + str(len(series)))

      if (len(series) > 0):
          df = pd.DataFrame.from_dict(series)
          print(len(df))
          df = df[(df.lon >= left) & (df.lon <= right) & (df.lat >= bottom) & (df.lat <= top)]
          print(len(df))
          df.name = series_beam['name']
          df_beams.append(df)

pd.concat(df_beams).to_csv(filepath)



In [ ]:
print(df_beams)
del df_beams

In [ ]:
elevation_data[0]['beam_name']